In [30]:
# ------------------------------------------------------------
# 1️⃣ 문서 로드 및 분할
# ------------------------------------------------------------
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Word(docx) 문서를 읽어오는 로더
loader = Docx2txtLoader("./tax_with_markdown.docx")

# 긴 문서를 LLM이 다루기 쉽도록 chunk로 쪼갠다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   # 한 조각(청크) 최대 길이(문자 기반; 토큰 유사)
    chunk_overlap=200, # 앞/뒤 청크끼리 겹쳐줄 길이 (문맥 보존)
)

# load_and_split로 바로 쪼개진 Document 리스트를 얻는다.
document_list = loader.load_and_split(text_splitter=text_splitter)

In [31]:
# ------------------------------------------------------------
# 2️⃣ 임베딩 & Pinecone에 벡터로 저장
# ------------------------------------------------------------
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_community.vectorstores import Pinecone as PineconeVectorStore

load_dotenv()  # .env에서 OPENAI_API_KEY, PINECONE_API_KEY 읽어옴

# 텍스트 → 벡터로 바꿔줄 임베딩 모델
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# Pinecone 초기화
index_name = "tax-index"
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# 문서들을 벡터화해서 Pinecone 인덱스에 넣는다.
# 주의: from_documents()의 첫 번째 파라미터는 documents= 여야 한다.
database = PineconeVectorStore.from_documents(
    documents=document_list,   # ✅ 정확한 이름
    embedding=embedding,
    index_name=index_name,
)

document_list[52]

Document(metadata={'source': './tax_with_markdown.docx', 'text': '제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한

In [50]:
# ------------------------------------------------------------
# 3️⃣ 질문 & 유사 문서 검색 (retrieval)
# ------------------------------------------------------------
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# # 질문과 가장 가까운 문서 chunk k개를 벡터DB에서 가져온다.
# retrieved_docs = database.similarity_search(query, k=3)

# # LLM이 이해할 수 있도록, 문서 내용만 꺼내 하나의 큰 문자열로 합친다.
# context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

# retrieved_docs

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

dictionary = {
    "직장인": "거주자",
    "근로자": "거주자",
    "월급": "근로소득",
    "세금": "소득세"
}

# ✅ f-string ❌  그냥 문자열로 유지해야 LangChain이 {변수}를 나중에 채움
prompt = ChatPromptTemplate.from_template("""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해주세요

사전: {dictionary}
질문: {question}
""")

dictionary_chain = prompt | llm | StrOutputParser()

new_query = dictionary_chain.invoke({
    "dictionary": dictionary,
    "question": query
})


In [63]:
print("🧩 교정된 질문:", new_query)

🧩 교정된 질문: 연봉 5천만원인 거주자의 소득세는 얼마인가요?


In [64]:
# 질문과 가장 가까운 문서 chunk k개를 벡터DB에서 가져온다.
retrieved_docs = database.similarity_search(new_query, k=3)

# LLM이 이해할 수 있도록, 문서 내용만 꺼내 하나의 큰 문자열로 합친다.
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [65]:
# ------------------------------------------------------------
# 4️⃣ 프프롬프트 템플릿 + LLM → "체인" 만들기
# ------------------------------------------------------------
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 실제로 답변을 생성할 LLM (대화형)
llm = ChatOpenAI(model="gpt-4o")

# PromptTemplate: LLM에 보낼 "서식"을 미리 만들어 둔다.
# {context}, {question} 자리에 나중에 실제 값이 들어감.
rag_prompt_template = PromptTemplate(
    template="""
    You are a helpful assistant that can answer questions about Korean tax law.
    Use the following context to answer **accurately and concisely in Korean language**.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """,
    input_variables=["context", "question"]
)

# ---- 체인(chain)이 뭐냐? ----
# rag_prompt_template | llm  는 “프롬프트를 만든 다음 그 결과를 LLM에 바로 넣어라”는 **연결**을 뜻함.
# 즉, PromptTemplate의 출력(완성된 문자열) → LLM의 입력 으로 곧장 넘어가는 파이프.
rag_chain = rag_prompt_template | llm
rag_chain

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    You are a helpful assistant that can answer questions about Korean tax law.\n    Use the following context to answer **accurately and concisely in Korean language**.\n\n    Context:\n    {context}\n\n    Question:\n    {question}\n\n    Answer:\n    ')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x3665bd0f0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x365e16f80>, root_client=<openai.OpenAI object at 0x3665bcbb0>, root_async_client=<openai.AsyncOpenAI object at 0x3665bcca0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [68]:
# ------------------------------------------------------------
# 5️⃣ 체인 실행: invoke
# ------------------------------------------------------------
# invoke는 “이 체인을 여기 입력값으로 한 번 실행해”라는 의미.
# dict를 넣으면 PromptTemplate가 {context}, {question}를 채우고,
# 그 결과 문자열을 LLM이 받아 답변을 생성한다.
ai_message = rag_chain.invoke({
    "context": context_text,
    "question": query
})
ai_message

AIMessage(content='연봉 5,000만원인 직장인의 소득세는 다음과 같이 계산할 수 있습니다.\n\n1. 연봉 5,000만원은 제55조의 종합소득 과세표준에 따라 "1,400만원 초과 5,000만원 이하" 구간에 해당됩니다.\n2. 이 구간의 세율은 "84만원 + (1,400만원을 초과하는 금액의 15퍼센트)"입니다.\n3. 1,400만원 초과 금액은 5,000만원 - 1,400만원 = 3,600만원입니다.\n4. 3,600만원의 15퍼센트는 3,600만원 × 0.15 = 540만원입니다.\n5. 따라서 종합소득산출세액은 84만원 + 540만원 = 624만원입니다.\n\n따라서, 연봉 5,000만원인 직장인의 소득세는 624만원입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 210, 'prompt_tokens': 2682, 'total_tokens': 2892, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2048}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_65564d8ba5', 'id': 'chatcmpl-CWNFlGOALkg4XNnC3i947XWWXKHEa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--753b5051-6a1a-4721-8b4b-0bb83734cbfc-0', usage

In [69]:
# ------------------------------------------------------------
# 6) 결과 출력 (질문/컨텍스트/답변)
# ------------------------------------------------------------
print("🧩 질문:")
print(query)

print("\n📚 검색된 컨텍스트:")
# 너무 길면 앞부분만 보자. 전부 보고 싶으면 그냥 context_text 출력.
print(context_text.replace("\n", "")[:1200], "...\n(생략)")

print("\n💬 답변:")
print(ai_message.content)

# 참고: retrieved_docs[i].metadata 로 파일 경로, 페이지 등 메타데이터도 볼 수 있음

🧩 질문:
연봉 5천만원인 직장인의 소득세는 얼마인가요?

📚 검색된 컨텍스트:
제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>| 종합소득 과세표준          | 세율                                         ||-------------------|--------------------------------------------|| 1,400만원 이하     | 과세표준의 6퍼센트                             || 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  || 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) || 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   || 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 

개념 설명 (체인, invoke를 이해하기 쉽게)
1) 체인(chain) = 연결된 단계들
“이 단계 결과를 다음 단계 입력으로 자동 전달해줘”
LangChain 최신 문법에선 파이프(|) 연산자로 연결해요.
rag_prompt_template | llm
→ “프롬프트(문자열) 만들어 → LLM에게 바로 줘서 답을 받아라”는 작업 파이프라인.
2) invoke = 한 번 실행
rag_chain.invoke({...})
→ 이 파이프라인을 주어진 입력으로 1회 실행.
비유: “빵 반죽(입력)을 넣으면, 체인(파이프라인)이 자동으로 반죽 → 발효 → 굽기 단계를 지나 **완성 빵(출력)**을 내놓는다.”
3) 자주 나오는 메서드 3형제
invoke(input_dict) : 단건 실행
batch([input_dict1, input_dict2, ...]) : 여러 건 일괄 실행
ainvoke(input_dict) : 비동기 한 건 실행 (await 필요)